# Day 10
## Part 1
Represent lights and buttons with sets and BFS. You could do this more efficiently with bitmasks as the lights and buttons are binary numbers.

In [1]:
from advent import read_input

def parse_data(s):
    data = []
    for line in s.strip().splitlines():
        fields = line.strip().split()
        lights = frozenset(
            i 
            for i, c in enumerate(fields[0][1:-1]) 
            if c == "#"
        )
        buttons = [
            frozenset(eval(field.replace(")", ",)")))
            for field in fields[1:-1]
        ]
        joltages = eval(fields[-1].replace("{", "[").replace("}", "]"))
        data.append((lights, buttons, joltages))
    return data

test_data = parse_data("""[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
""")

test_data

[(frozenset({1, 2}),
  [frozenset({3}),
   frozenset({1, 3}),
   frozenset({2}),
   frozenset({2, 3}),
   frozenset({0, 2}),
   frozenset({0, 1})],
  [3, 5, 4, 7]),
 (frozenset({3}),
  [frozenset({0, 2, 3, 4}),
   frozenset({2, 3}),
   frozenset({0, 4}),
   frozenset({0, 1, 2}),
   frozenset({1, 2, 3, 4})],
  [7, 5, 12, 7, 2]),
 (frozenset({1, 2, 3, 5}),
  [frozenset({0, 1, 2, 3, 4}),
   frozenset({0, 3, 4}),
   frozenset({0, 1, 2, 4, 5}),
   frozenset({1, 2})],
  [10, 11, 11, 5, 10, 5])]

In [2]:
from collections import deque

def min_presses(lights, buttons):
    q = deque([(frozenset(), 0)])
    seen = {frozenset()}
    while q:
        l, n = q.popleft()
        if l == lights:
            return n
        for b in buttons:
            new_ls = l ^ b
            if new_ls not in seen:
                q.append((new_ls, n + 1))
                seen.add(new_ls)

[min_presses(ls, bs) for ls, bs, _ in test_data]

[2, 3, 2]

In [3]:
def part_1(data):
    return sum(min_presses(ls, bs) for ls, bs, _ in data)

assert part_1(test_data) == 7

In [4]:
data = parse_data(read_input())

part_1(data)

484

## Part 2

Good old dependable dynamic programming has failed me here, working on the test data but failing on the first instance in the real data.

These are a set of equations, where a given joltage is the sum of the number of buttons affecting that joltage is pressed. We would like to solve for each button's number of presses but from attempting this on paper this is not always possible. So simplify the equations and then search from there.

In [93]:
from collections import Counter
from itertools import chain, combinations, permutations

def eq_str(eq):
    joltage, ns = eq
    return f"{joltage} = {' + '.join('n' + str(n) for n in ns)}"

def simplify_eqs(buttons, joltages):
    eqs = []
    for i, joltage in enumerate(joltages):
        ns = {bi for bi, b in enumerate(buttons) if i in b}
        eqs.append([joltage, ns])
        # print(eq_str((joltage, ns)))
    ns = {}

    def simplify():
        # Remove duplicate equations
        for i, j in combinations(range(len(eqs)), 2):
            if eqs[i] == eqs[j]:
                # print(f"Removing duplicate: {eq_str(eqs[j])}")
                del eqs[j]
                return True
        # Resolve equations with single variable
        for i, (joltage, bs) in enumerate(eqs):
            if len(bs) == 1:
                # print(f"Resolved: {eq_str(eqs[i])}")
                x = bs.pop()
                ns[x] = joltage
                del eqs[i]
                for i in range(len(eqs)):
                    if x in eqs[i][1]:
                        eqs[i][1].remove(x)
                        eqs[i][0] -= ns[x]
                return True
        # If a joltage is zero, set all to zero
        for i, (joltage, bs) in enumerate(eqs):
            if joltage == 0:
                print(f"All zero: {eq_str(eqs[i])}")
                for b in bs:
                    eqs.append([0, {b}])
                del eqs[i]
                return True
        # Resolve pairs of equations where one has an extra variable
        for i, j in permutations(range(len(eqs)), 2):
            if eqs[j][1] - eqs[i][1] == set():
                diff = eqs[i][1] - eqs[j][1]
                if len(diff) == 1:
                    x = (eqs[i][1] - eqs[j][1]).pop()
                    xjolt = eqs[i][0] - eqs[j][0]
                    # print(f"Worked out: {eq_str(eqs[i])} - {eq_str(eqs[j])}")
                    eqs.append([xjolt, {x}])
                    return True
                else: 
                    # print(f"Diffing: {eq_str(eqs[i])} - {eq_str(eqs[j])}")
                    eqs.append([eqs[i][0] - eqs[j][0], diff])
                    del eqs[i]
                    return True
        # If a variable is in all equations set it to the minimum joltage
        for i in range(len(joltages)):
            if len(eqs) > 1 and all(i in eq[1] for eq in eqs):
                min_joltage = min(eq[0] for eq in eqs)
                # print(f"All equations contain {i}, setting to {min_joltage}")
                eqs.append([min_joltage, {i}])
                return True
        return False
        
                
    while simplify():
        pass

    return eqs, ns

_, bs, js = test_data[1]
simplify_eqs(bs, js)

([], {4: 0, 3: 5, 0: 2, 2: 0, 1: 5})

In [94]:
completely_resolved = 0
partially_resolved = 0
unresolved = 0
for i, (_, bs, js) in enumerate(data):
    eqs, ns = simplify_eqs(bs, js)
    if not eqs:
        completely_resolved += 1
    elif ns:
        partially_resolved += 1
    else:
        unresolved += 1
print(len(data))
print(completely_resolved)
print(partially_resolved)
print(unresolved)

All zero: 0 = n3 + n5
All zero: 0 = n1 + n6
186
70
76
40


In [96]:
_, bs, js = data[1]
simplify_eqs(bs, js)

([[146, {1, 2}], [141, {2, 3}], [5, {1, 3}]], {0: 11})